In [95]:
import time
import numpy as np
import pandas as pd
import peakutils as pu
from scipy import signal

In [37]:
class AV_info:
    def __init__(self, filename):
        self.stimuli_duration = 50
        self.rest_interval = 15
        self.startsecs = 0
        self.PID = ''
        self.stimulus_order = []
        self.stimulus_info = ['native_w_touch', 'native_w/o_touch',
                              'nonnative_w_touch', 'nonnative_w/o_touch',
                              'native_async_w_touch', 'native_async_w/o_touch']
        with open(filename, 'r') as f:
            self.lines = f.readlines()
    def get_start_secs(self):
        t = self.lines[1].split()
        t = ':'.join(t[0].split(':')[:3])
        d = self.lines[16].split()[2]
        dt = d + ' ' + t
        self.startsecs = time.mktime(time.strptime(dt, '%m/%d/%Y %H:%M:%S'))
    def get_PID(self):
        self.PID = self.lines[15].split()[2]
    def get_order(self):
        order = self.lines[18].split()[3:]
        self.stimulus_order = [int(o) for o in order]
    def process(self):
        self.get_start_secs()
        self.get_PID()
        self.get_order()

In [85]:
class Physio_data:
    def __init__(self, startsecs, PID):
        self.startsecs = startsecs
        with open(PID+'/EDA.csv', 'r') as f:
            EDA_lines = f.readlines()
        with open(PID+'/BVP.csv', 'r') as f:
            BVP_lines = f.readlines()
        with open(PID+'/HR.csv', 'r') as f:
            HR_lines = f.readlines()
        with open(PID+'/IBI.csv', 'r') as f:
            self.IBI_lines = f.readlines()
        self.EDA_ = np.array([float(d.strip()) for d in EDA_lines])
        self.BVP_ = np.array([float(d.strip()) for d in BVP_lines])
        self.HR_ = np.array([float(d.strip()) for d in HR_lines])
        self.IBI_ = pd.read_csv(PID+'/IBI.csv',skiprows=[0],names=['time','ibi'])
    def get_EDA(self):
        start_time = self.EDA_[0]
        freq = int(self.EDA_[1])
        offset = int(self.startsecs-start_time)
        EDA_ = self.EDA_[offset+2:]
        self.EDA = [EDA_[(50+15)*freq*i:(50+15)*freq*i+50*freq] for i in range(6)]
    def get_BVP(self):
        start_time = self.BVP_[0]
        freq = int(self.BVP_[1])
        offset = int(self.startsecs-start_time)
        BVP_ = self.BVP_[offset+2:]
        self.BVP = [BVP_[(50+15)*freq*i:(50+15)*freq*i+50*freq] for i in range(6)]
    def get_HR(self):
        start_time = self.HR_[0]
        freq = int(self.HR_[1])
        offset = int(self.startsecs-start_time)
        HR_ = self.HR_[offset+2:]
        self.HR = [HR_[(50+15)*freq*i:(50+15)*freq*i+50*freq] for i in range(6)]
    def get_IBI(self):
        start_time = float(self.IBI_lines[0].split(',')[0])
        offset = int(self.startsecs-start_time)
        time_intervals = [(offset+(50+15)*i, offset+(50+15)*i+50) for i in range(6)]
        self.IBI = [[] for _ in range(6)]
        for i in range(6):
            interval = time_intervals[i]
            trial = self.IBI_[self.IBI_.time<interval[1]][self.IBI_.time>interval[0]]
            self.IBI[i] = trial.ibi.values
    def process(self):
        self.get_EDA()
        self.get_BVP()
        self.get_HR()
        self.get_IBI()

In [101]:
class Physio_features:
    def __init__(self):
        self.feature_names = ['hr', 'ibi', 'scl_mean', 'scl_std',
                              'scr_rate', 'scr_mean', 'scr_std', 'scr_max']
        self.features = [[] for _ in range(6)]
    def extract_bvp_fv(self, hrs, ibis):
        for i in range(6):
            self.features[i].append(np.mean(hrs[i]))
            self.features[i].append(np.mean(ibis[i]))
    def extract_eda_fv(self, edas):
        for i in range(6):
            eda = edas[i]
            scr,scl = self.decompose_eda(eda)
            self.features[i].append(np.mean(scl))
            self.features[i].append(np.std(scl))
            for fv in self.compute_scr_fv(scr):
                self.features[i].append(fv)
    def decompose_eda(self, eda):
        """Decompose EDA signal into SCR and SCL respectively."""
        if len(eda)<20:
            return 'NA','NA'
        b,a = signal.butter(4,0.5/2)
        gsr_filt = signal.filtfilt(b,a,eda)
        b,a = signal.butter(4,0.05/2,'highpass')
        scr = signal.filtfilt(b,a,gsr_filt)
        scl = [x-y for x,y in zip(gsr_filt,scr)]
        return scr,scl
    def compute_scr_fv(self, scr):
        #peaks = signal.find_peaks_cwt(scr_lp,np.arange(1,20))
        if scr == 'NA':
            return 'NA','NA','NA','NA'

        peaks = pu.indexes(scr,0.6,15)
        t = len(scr)/float(4*60)
        scr_rate = len(peaks)/t

        responses = [scr[i] for i in peaks]
        scr_mean = np.mean(responses)
        scr_sd = np.std(responses)
        scr_max = np.max(responses)
        return scr_rate, scr_mean, scr_sd, scr_max

In [99]:
physioData

In [102]:
feature_test = Physio_features()

In [103]:
feature_test.extract_bvp_fv(physioData.HR,physioData.IBI)
feature_test.extract_eda_fv(physioData.EDA)

C:\Users\biand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [104]:
feature_test.feature_names

['hr',
 'ibi',
 'scl_mean',
 'scl_std',
 'scr_rate',
 'scr_mean',
 'scr_std',
 'scr_max']

In [105]:
feature_test.features

[[171.4194,
  0.388462546511628,
  0.128116792045257,
  0.00479168589552,
  7.199999999999999,
  0.0013393513960035105,
  0.0004737418601264578,
  0.00234067515437222],
 [158.70420000000001,
  0.38080442592592584,
  0.14990170995979593,
  0.004588525675728073,
  9.6,
  0.0011098420805003895,
  0.0005326472392542174,
  0.002206878544793829],
 [155.1818,
  0.41913682352941173,
  0.16912669760129628,
  0.004453271095625084,
  9.6,
  0.0013326492666581933,
  0.0003919540909926174,
  0.002200998620443187],
 [164.1764,
  0.41033554794520544,
  0.18645284177271118,
  0.0039645430451851135,
  8.4,
  0.0012540538534380147,
  0.00039604661380679087,
  0.0017487519925772376],
 [134.9242,
  0.48211835416666665,
  0.20302161711659694,
  0.0034304353249951227,
  9.6,
  0.0008396170663348584,
  0.0005124505154166972,
  0.001497079207713209],
 [129.5772,
  0.46564627499999994,
  0.21906771734682104,
  0.003534262968227336,
  8.4,
  0.0009580637776001879,
  0.0009477051655013599,
  0.002515667199659639